# Clock Domains

一个新的synchronous control domain， 通常称为clock domain，可以通过创建一个ClockDomain对象并将其添加到m.domains

注意：m.d是m.domains.sync的语法糖（快捷写法），用于默认同步域下定义的同步逻辑。

m.domain是个容器对象，用于管理设计中的所有时钟域（如sync，pix，usb等）每个域需要单独定义的时钟和复位信号。

In [ ]:
m.domains.video = cd_video = ClockDomain()

如果事先不知道domain的名称，则可以使用另一种不太简洁的语法：

In [ ]:
def add_video_domain(n):
    cd = ClockDomain(f"video_{n}")
    m.domains += cd
    return cd

add_video_domain(2)

每当使用domain_name = ClockDomain(...)或m.domains.domain_name = ClockDomain(...)

语法立即分配创建的ClockDomain对象时，ClockDomain()调用中可能会省略domain名称。在其他情况下，它必须作为地一个参数提供。

clock domain总是有一个clock信号，可以通过cd.clk属性访问。

默认情况下，clock domain是上升沿有效，也可以配置为下降沿有效。

In [ ]:
m.domain.jtag = ClockDomain(clk_edge="neg")

clock domain也有一个reset信号，可以通过cd.rst属性访问。

reset信号始终为高电平有效：如果reset信号的值为1,则clock domain中的信号被reset。

该信号的初始值为0, 因此如果从未分配reset信号，则clock domain中的信号永远不会显式reset（power-on reset）。

如果不需要复位，可在clock domain构造的时按照如下配置

In [ ]:
m.domains.startup = ClockDomain(reset_less=True)

在reset-less的clock domain中的信号，让然可以使用ResetInserter control flow修饰符显式reset。

## clock和reset信号绑定

Clock domain是late bound（延迟绑定）的，这意味着在创建具有该名称的ClockDomain对象并将其添加到design之前，

可以使用domain名称引用domain的clock和reset信号。

In [ ]:
m.d.comb += [
    ClockSignal().eq(bus_clk),
    ResetSignal().eq(~bus_rstn)
]

在该示例中，一旦design被处理，在此module或其包含的modules之一中找到clock domain sync的clock信号等于bus_clk。

同一clock domain的reset信号将等于被否定的bus_rstn。

在sync domain模块中创建

In [ ]:
m.domains.sync = cd_sync = ClockDomain()
m.d.comb += [
    cd_sync.clk.eq(bus_clk),
    cd_sync.rst.eq(~bus_rstn),
]

ClockSignal和ResetSignal又可以分配给其他信号并在表达式中使用。

他们采用单个参数，即domain的名称，如果未指定，则默认为sync。

在表达式中使用ClockSignal或cd.clk时要特别小心。

与clock信号进行assign通常是安全的，任何其他操作可能产生不可预知的结果。

请查阅synthesis toolchain和platform的文档，以了解允许使用clock signal的作。

FPGAs通常具有专用的clocking设施，可用于禁用，分品或多路服用clock signals。

当以FPGA为目标时，应尽可能使用这些工具，

并且应避免使用像ClockSingal() & en或Mux(sel, ClockSignal("a"), ClockSignal("b"))